In [176]:
import pandas as pd
import numpy as np
import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0 seconds ---


In [177]:
start_time = time.time()
#Phase1 Using the offline workload A as the traning data

#Step1 Load data

workload_A=pd.read_csv('../../offline_workload.csv') 
#print(workload_A.head())

print("--- %s seconds ---" % (time.time() - start_time))

--- 1.468073844909668 seconds ---


In [178]:
start_time = time.time()

#Step2 Split workload_A by creating different dataframes

workload_A_ids = workload_A["workload id"].unique()
workload_A_list = []
for num,workload_A_id in enumerate(workload_A_ids,start=1):
    tmp_workload = workload_A[workload_A['workload id'].isin([workload_A_id])]
    workload_A_list.append(tmp_workload)
    exec("workload_A_%s = tmp_workload"%num)

#workload_A_1
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.09574437141418457 seconds ---


In [179]:
start_time = time.time()

#Step3 Get metrics

tmpframe = workload_A.T.copy()
tmpframe = tmpframe.tail(585-1-12).T
metrics_name = tmpframe.columns
#print(metrics_name)
workload_A_listformetrics = []
num_workloads_A = len(workload_A_list)
for i in range(num_workloads_A):
    tmp = (workload_A_list[i])[metrics_name]
    workload_A_listformetrics.append(tmp)

print("--- %s seconds ---" % (time.time() - start_time))


--- 2.4664032459259033 seconds ---


In [180]:
start_time = time.time()

#Step4 FA (apply for all workloads)
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import normalize

transformer = FactorAnalysis(n_components=7, random_state=0)
workload_list_afterFA = []

for i in range(58):
    tmp_workload_A_listformetrics = workload_A_listformetrics[i]
    tmp_workload_A_listformetrics_array = tmp_workload_A_listformetrics.values
    
    tmp_workload_A_listformetrics_array_normalized = normalize(tmp_workload_A_listformetrics_array, norm='l2')
    
    #tmp_workload_A_listformetrics_array_Trans = tmp_workload_A_listformetrics_array.T
    tmp_workload_A_listformetrics_array_Trans_normalized = tmp_workload_A_listformetrics_array_normalized.T
    
    tmp_transformer = FactorAnalysis(n_components=7, random_state=0)
    #print(tmp_workload_A_listformetrics_array_Trans)
    #normalize tmp_workload_A_listformetrics_array_Trans
    #tmp_workload_A_listformetrics_array_Trans_normalized = normalize(tmp_workload_A_listformetrics_array_Trans, norm='l2')
    #print('-------------------------')
    #print(tmp_workload_A_listformetrics_array_Trans_normalized)
    
    tmp_workload_A_listformetrics_array_Trans_transformed = transformer.fit_transform(tmp_workload_A_listformetrics_array_Trans_normalized)
    workload_list_afterFA.append(tmp_workload_A_listformetrics_array_Trans_transformed)

print("--- %s seconds ---" % (time.time() - start_time))


--- 12.56239938735962 seconds ---


In [181]:
start_time = time.time()

#Step5 K-means (apply for all workloads)
from sklearn.cluster import KMeans


workload_list_centers_metrics_name = []

for i in range(58):
    tmp_workload_A_listformetrics_array_Kmeans = KMeans(n_clusters=2, random_state=0).fit(workload_list_afterFA[i])
    tmp_cluster_centers = tmp_workload_A_listformetrics_array_Kmeans.cluster_centers_
    tmp_centers_index = np.where((workload_list_afterFA[i]==tmp_cluster_centers[:,None]).all(-1))
    tmp_centers_metrics_name = []
    tmp_n_centers = len(tmp_centers_index)
    for j in range(tmp_n_centers):
        tmp_mname = metrics_name[tmp_centers_index[j][0]]
        tmp_centers_metrics_name.append(tmp_mname)
    
    workload_list_centers_metrics_name.append(tmp_centers_metrics_name)

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.5375630855560303 seconds ---


In [182]:
start_time = time.time()

#Step6 Get Prutiend workload (apply for all workloads)

knobs_latency_list = ['k1','k2','k3','k4','k5','k6','k7','k8','s1','s2','s3','s4']
latency = ['latency']

pruning_workload_list = []

for i in range(num_workloads_A):
    tmp_pruning_list = knobs_latency_list + latency + workload_list_centers_metrics_name[i]
    tmp_pruning_workload_A = (workload_A_list[i])[tmp_pruning_list]
    pruning_workload_list.append(tmp_pruning_workload_A)

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.038915157318115234 seconds ---


In [183]:
start_time = time.time()

#Step7 Building GPR (apply for all workloads)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler


tmp_kernel = 1.0 * RBF(length_scale=1.0)

workload_A_GPRmodels_lists = []
workload_A_scalar_lists = []
workload_A_normalized_m = []

for i in range(58):
    tmp_GPRmodels_lists = []
    tmp_scalar_lists = []
    tmp_normalized_m = []
    
    tmp_Pruning_workload_A_X = ((workload_A_list[i])[knobs_latency_list]).values
    tmp_Pruning_workload_A_y_o = ((workload_A_list[i])[latency]).values
    #print(tmp_Pruning_workload_A_y_o)
    
    tmp_x_scaler = MinMaxScaler().fit(tmp_Pruning_workload_A_X)
    tmp_y_o_scaler = MinMaxScaler().fit(tmp_Pruning_workload_A_y_o)
    
    tmp_scalar_lists.append(tmp_x_scaler)
    tmp_scalar_lists.append(tmp_y_o_scaler)
    
    tmp_Pruning_workload_A_X_normalized = tmp_x_scaler.transform(tmp_Pruning_workload_A_X)
    tmp_Pruning_workload_A_y_o_normalized = tmp_y_o_scaler.transform(tmp_Pruning_workload_A_y_o)
    
    tmp_normalized_m.append(tmp_Pruning_workload_A_y_o_normalized)
    #print(tmp_Pruning_workload_A_y_o_normalized)
    
    #tmp_kernel = DotProduct() + WhiteKernel()
    tmp_workload_A_gpr_o = GaussianProcessRegressor(kernel=tmp_kernel,random_state=0).fit(tmp_Pruning_workload_A_X_normalized, tmp_Pruning_workload_A_y_o_normalized)
    
    tmp_GPRmodels_lists.append(tmp_workload_A_gpr_o)
    
    tmp_num_centers = len(workload_list_centers_metrics_name[i])
    for j in range(tmp_num_centers):
        
        tmp_Pruning_workload_A_y_m = ((workload_A_list[i])[[workload_list_centers_metrics_name[i][j]]]).values
        #print(tmp_Pruning_workload_A_y_m)
        tmp_y_m_scaler = MinMaxScaler().fit(tmp_Pruning_workload_A_y_m)
        tmp_scalar_lists.append(tmp_y_m_scaler)
        tmp_Pruning_workload_A_y_m_normalized = tmp_y_m_scaler.transform(tmp_Pruning_workload_A_y_m)
        tmp_normalized_m.append(tmp_Pruning_workload_A_y_m_normalized)
        
        #tmp_kernel = DotProduct() + WhiteKernel()
        tmp_workload_A_gpr_m = GaussianProcessRegressor(kernel=tmp_kernel,random_state=0).fit(tmp_Pruning_workload_A_X_normalized, tmp_Pruning_workload_A_y_m_normalized)
        tmp_GPRmodels_lists.append(tmp_workload_A_gpr_m)
        
    workload_A_GPRmodels_lists.append(tmp_GPRmodels_lists)
    workload_A_scalar_lists.append(tmp_scalar_lists)
    workload_A_normalized_m.append(tmp_normalized_m)
print("--- %s seconds ---" % (time.time() - start_time))
#print(workload_A_normalized_m)

--- 24.631118297576904 seconds ---


In [184]:
#*************************************************************************************************************************************************
#    1
#    We have workload_A_list contain 58 workloads
#    workload_A_list
#
#    2
#    We have workload_list_centers_metrics_name contain 58 lists, each list have the metrics kept after FA and K-means for each workload
#    workload_list_centers_metrics_name
#
#    3
#    We have pruning_workload_list contain 58 workloads after prunning
#    pruning_workload_list
#
#    4
#    We have workload_A_GPRmodels_lists contain 58 lists, each list have 1(latency) + k(number of metrics kept) GPR models
#    workload_A_GPRmodels_lists
#
#*************************************************************************************************************************************************

In [185]:
#Phase2 Using the online workload B as the validation data

#In workloads B, each workload has 6 data points, we use 5 of them apply workload mapping, and use the left 1 as the target workload, predict
#latency and compare it with the actual value in workloads B.

In [186]:
start_time = time.time()

#Step1 load workload B data set

workload_B=pd.read_csv('../../online_workload_B.csv') 

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.09574484825134277 seconds ---


In [187]:
start_time = time.time()

#Step2 Split workload_B by creating different dataframes

workload_B_ids = workload_B["workload id"].unique()
workload_B_list = []
for num,workload_B_id in enumerate(workload_B_ids,start=1):
    tmp_workload = workload_B[workload_B['workload id'].isin([workload_B_id])]
    workload_B_list.append(tmp_workload)
    exec("workload_B_%s = tmp_workload"%num)

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.07878923416137695 seconds ---


In [188]:
start_time = time.time()

#Step3 Split each workload into 5 data points and 1 data point as target

num_workloads_B = len(workload_B_list)

workload_B_mapping_data = []
workload_B_target = []

for i in range(num_workloads_B):
    workload_B_mapping_data.append(workload_B_list[i][:-1])
    workload_B_target.append(workload_B_list[i][5:6])
    
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.03889584541320801 seconds ---


In [189]:
start_time = time.time()

#Step4 workload mapping Test for all workload in workload B
all_distance_list = []
for i in range(100):
    tmp_test_workload_B_mapping = workload_B_mapping_data[i]
    tmp_test_workload_B_mapping_cfigs = ((tmp_test_workload_B_mapping)[knobs_latency_list]).values
    tmp_distance_list = []
    
    for j in range(58):
        tmp_test_workload_A = workload_A_list[j]
        tmp_test_workload_A_metrics = workload_list_centers_metrics_name[j]
        tmp_test_workload_A_gpr_model = workload_A_GPRmodels_lists[j]
        tmp_test_prunning_workload_A = pruning_workload_list[j]
        tmp_test_workload_A_cigs_scalar = workload_A_scalar_lists[j][0]
        tmp_test_workload_A_normalized_m = workload_A_normalized_m[j]
        
        tmp_norm_workload_B_mapping_cfigs = tmp_test_workload_A_cigs_scalar.transform(tmp_test_workload_B_mapping_cfigs)
        #print(tmp_norm_workload_B_mapping_cfigs)
        
        tmp_test_workload_A_cfigs = ((tmp_test_workload_A)[knobs_latency_list]).values
    
        tmp_corr_metric_values = []
        
        for k in range(5):
            tmp_data_point = tmp_test_workload_B_mapping_cfigs[k]
            tmp_check = np.where((tmp_data_point==tmp_test_workload_A_cfigs[:,None]).all(-1))
            
            if len(tmp_check[0]) == 0:
                tmp_gpr_predic = []
                for m in range(2):
                    tmp_gpr_predic.append(tmp_test_workload_A_gpr_model[m+1].predict([tmp_norm_workload_B_mapping_cfigs[k]])[0])
                tmp_corr_metric_values.append(tmp_gpr_predic)
                #print(tmp_gpr_predic)
            
            else:
                tmp_gpr_predic = []
                tmp_row_idx = tmp_check[0][0]
                #print(tmp_row_idx)
                #print('xxxxxxxxxxxxxxxxxxx')
                tmp_test_prunning_workload_A_array = tmp_test_prunning_workload_A.values
                #print(tmp_test_workload_A_normalized_m)
                for n in range(2):
                    #tmp_gpr_predic.append(tmp_test_prunning_workload_A_array[tmp_row_idx][13+n])
                    tmp_gpr_predic.append(tmp_test_workload_A_normalized_m[n+1][tmp_row_idx])
                    
                #tmp_gpr_predic.append(tmp_test_prunning_workload_A_array[tmp_row_idx][13])
                #tmp_gpr_predic.append(tmp_test_prunning_workload_A_array[tmp_row_idx][14])
                tmp_corr_metric_values.append(tmp_gpr_predic)
        
        tmp_test_workload_B_mapping_metrics_norm_value_list = []
        for m in range(2):
            tmp_test_workload_B_mapping_metrics_value = tmp_test_workload_B_mapping[[tmp_test_workload_A_metrics[m]]].values
            tmp_test_workload_B_mapping_metrics_nomr_value = workload_A_scalar_lists[j][2+m].transform(tmp_test_workload_B_mapping_metrics_value)
            tmp_test_workload_B_mapping_metrics_norm_value_list.append(tmp_test_workload_B_mapping_metrics_nomr_value)
            
            
        #tmp_test_workload_B_mapping_metrics_norm_value_arry = np.asarray(tmp_test_workload_B_mapping_metrics_norm_value_list).reshape((2,5))
        #tmp_corr_metric_values_arry = np.asarray(tmp_corr_metric_values).reshape((2,5))
        #print(tmp_test_workload_B_mapping_metrics_norm_value_list)
        
        #print('---')
        #print(tmp_corr_metric_values)
        #print('xxxxxxxxxxxxxx')
        tmp_test_workload_B_mapping_metrics_norm_value_arry = np.asarray(tmp_test_workload_B_mapping_metrics_norm_value_list).reshape((2,5))       
        tmp_corr_metric_values_arry = np.asarray(tmp_corr_metric_values).reshape((5,2)).T
        #print(tmp_test_workload_B_mapping_metrics_norm_value_arry)
        #print('-----')
        #print(tmp_corr_metric_values_arry)
        
        
        #tmp_test_workload_B_mapping_metrics_value = tmp_test_workload_B_mapping[tmp_test_workload_A_metrics].values
        #print(tmp_test_workload_B_mapping_metrics_value)
        #print('--------')
        tmp_distance = np.sum((tmp_test_workload_B_mapping_metrics_norm_value_arry-tmp_corr_metric_values_arry)*(tmp_test_workload_B_mapping_metrics_norm_value_arry-tmp_corr_metric_values_arry))
        tmp_distance_list.append(tmp_distance)
     
    all_distance_list.append(tmp_distance_list)


    
all_min_idx_list = []
for i in range(100):
    tmp_idx = all_distance_list[i].index(min(all_distance_list[i]))
    all_min_idx_list.append(tmp_idx)

print("--- %s seconds ---" % (time.time() - start_time))


--- 19.093955278396606 seconds ---


In [ ]:
start_time = time.time()

#Step5 Concatenate trances from the observed target workload and the nearest neighbor workload for all workload B

predictGPRmodel_list = []
predictScalar_list = []
inputScalar_list = []

for i in range(100):
    tmp_similar_idx = all_min_idx_list[i]
    tmp_similar_workload = pruning_workload_list[tmp_similar_idx]
    tmp_similar_workload_configs = ((pruning_workload_list[tmp_similar_idx])[knobs_latency_list]).values
    tmp_similar_workload_metrics = ((pruning_workload_list[tmp_similar_idx])[latency + workload_list_centers_metrics_name[tmp_similar_idx]]).values
    tmp_workload_B_mapping_metrics_with_similar = ((workload_B_mapping_data[i])[latency + workload_list_centers_metrics_name[tmp_similar_idx]]).values
    
    tmp_Augmented_pruned_target_configs = np.array(tmp_similar_workload_configs)
    tmp_Augmented_pruned_target_metrics = np.array(tmp_similar_workload_metrics)
    
    tmp_workload_B_configs = workload_B_mapping_data[i][knobs_latency_list].values
    #print(tmp_workload_B_configs)
    
    for j in range(5):
        tmp_test_workload_B_mapping_cfigs = tmp_workload_B_configs[j]
        tmp_test_workload_B_mapping_metrics_with_similar = tmp_workload_B_mapping_metrics_with_similar[j]
        tmp_check = np.where((tmp_test_workload_B_mapping_cfigs==tmp_similar_workload_configs[:,None]).all(-1))
        
        if len(tmp_check[0]) == 0:
            tmp_Augmented_pruned_target_configs = np.insert(tmp_Augmented_pruned_target_configs,0,tmp_test_workload_B_mapping_cfigs,axis=0)
            tmp_Augmented_pruned_target_metrics = np.insert(tmp_Augmented_pruned_target_metrics,0,tmp_test_workload_B_mapping_metrics_with_similar,axis=0)
        else:
        
            tmp_row_idx = tmp_check[0][0]
            #print(tmp_row_idx)
            #print('xxxxxxxxxxxxxxxxxxx')
            tmp_Augmented_pruned_target_metrics[tmp_row_idx] = tmp_test_workload_B_mapping_metrics_with_similar
    
    tmp_metrics_o_size = tmp_Augmented_pruned_target_metrics.T[0].shape[0]
    tmp_nor_tmp_Augmented_pruned_target_metrics_o = tmp_Augmented_pruned_target_metrics.T[0].reshape((tmp_metrics_o_size,1))
    tmp_o_scalar = MinMaxScaler().fit(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    predictScalar_list.append(tmp_o_scalar)
    
    tmp_Augmented_pruned_target_metrics_nomr = tmp_o_scalar.transform(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    
    tmp_X_scalar = MinMaxScaler().fit(tmp_Augmented_pruned_target_configs)
    inputScalar_list.append(tmp_X_scalar)
    
    tmp_Augmented_pruned_target_configs_norm = tmp_X_scalar.transform(tmp_Augmented_pruned_target_configs)
    #print(tmp_Augmented_pruned_target_metrics_nomr)
    
    #print(tmp_Augmented_pruned_target_configs)
    #print('------')
    #print(tmp_nor_tmp_Augmented_pruned_target_metrics_o)
    
    test_workload_B_gpr_o = GaussianProcessRegressor(kernel=tmp_kernel,random_state=0).fit(tmp_Augmented_pruned_target_configs_norm, tmp_Augmented_pruned_target_metrics_nomr)
    predictGPRmodel_list.append(test_workload_B_gpr_o)
    
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
start_time = time.time()

#Step6 predict for all workload B

for i in range(30):
    tmp_target_config = workload_B_target[i][knobs_latency_list].values
    #print(tmp_target_config)
    tmp_target_config_norm = inputScalar_list[i].transform(tmp_target_config)
    #print(tmp_target_config_norm)
    tmp_max = (predictScalar_list[i].data_max_[0])
    tmp_min = (predictScalar_list[i].data_min_[0])
    tmp_predict = predictGPRmodel_list[i].predict(tmp_target_config_norm)
    tmp_predict_re = tmp_predict * (tmp_max - tmp_min) + tmp_min
    print('Predict: ',tmp_predict_re)
    print('Ground Truth: ',workload_B_target[i][latency].values)
    print('------------')
    

print("--- %s seconds ---" % (time.time() - start_time))
